### 配置文件验证

In [ ]:
from omegaconf import OmegaConf
hparams = OmegaConf.load('config.yaml') # 读取配置文件

### dataloader验证

In [1]:
import torch
from dataModule.dataset1 import FlatDataset
from torch.utils.data import DataLoader

In [2]:
dataset = FlatDataset('Spam_test.csv')
train_dataloader = DataLoader(dataset, 
                             batch_size=10, 
                             collate_fn=dataset.collater, 
                             shuffle=True,
                             num_workers=2)
batch = None
for i in train_dataloader:
    batch = i
    break

In [5]:
batch.keys()

dict_keys(['src_data', 'normal_data', 'miss_data', 'miss_matrix'])

### 模型精度测定

In [10]:
from dataModule.dataset1 import FlatDataset
from modelModule.model1 import VAE
from torch.utils.data import DataLoader
from utils import restore_data, result_show, get_missing
import torch

In [12]:
## 模型参数载入
model = VAE(dim=16, nhead=4)
checkpoint_path = '模型参数保存/Letter_lr_0_0001_lr_scheduler_CosineAnnealingLR/version_2/checkpoints/last.ckpt'    
checkpoint = torch.load(checkpoint_path)
model.load_state_dict({k.replace('model.',''):v for k,v in checkpoint['state_dict'].items()}) 
model.eval()
print('载入成功')

载入成功


In [18]:
dataset = FlatDataset('Letter_train.csv', is_test=True, data_norm='minmax_norm') # minmax_norm
index = 102
# src_data, M_matrix, miss_data = dataset[index]['src_data'].reshape(1,-1), dataset[index]['global_miss_matrix'].reshape(1,-1), dataset[index]['global_miss_data'].reshape(1,-1)
src_data = dataset[index]['src_data'].reshape(1,-1)
normal_data = dataset[index]['global_normal_data'].reshape(1,-1)
miss_data, M_matrix = get_missing(normal_data, 0.3) # 产生掩码数据，并采用随机数填补

imputed_data, _, _ = model(torch.from_numpy(miss_data).float(), torch.from_numpy(M_matrix).float()) 
imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy() # 得到最终插补结果,数据由[0,1]范围内
res_imputed_data = restore_data(imputed_data, dataset.Max_Val, dataset.Min_Val)
result_show(src_data, res_imputed_data, M_matrix)

原始数据 [ 6.  2.  6. 10.  9.]
插补数据 [5.3713654  3.5199894  4.69653413 7.89741933 8.36493373]


array([0.10477243, 0.7599947 , 0.21724431, 0.21025807, 0.07056292])